In [ ]:
import dv_processing as dv
import numpy as np
import os
# Sample VGA resolution, same as the DVXplorer camera
resolution = (32, 32)

# Event only configuration
#config = dv.io.MonoCameraWriter.EventOnlyConfig("DVXplorer_sample", resolution)

config = dv.io.MonoCameraWriter.Config("DVXplorer_sample")

# Add an event stream with a resolution
config.addEventStream(resolution)

# Add frame stream with a resolution
config.addFrameStream(resolution)

# Add IMU stream
config.addImuStream()

# Add trigger stream
config.addTriggerStream()

file_name = "spikesA"

# Create the writer instance, it will only have a single event output stream.
writer = dv.io.MonoCameraWriter(file_name + ".aedat4", config)

# Read text data line by line and parse events
events_list = []
events_store = dv.EventStore()

file_location = os.path.join(r"P:\NewLoihiData\Loihi_dataset", file_name+".log")

c = 0
timestamps = []

with open(file_location, "r") as f:
#with open("spikes_output_loihi.log", "r") as f:
    for idx, line in enumerate(f):
        timestamp_str, x_str, y_str, polarity_str = line.strip().split(",")
        timestamp = int(float(timestamp_str) * 1_000_000)  # Convert seconds to microseconds
        if idx==0:
            first_timestamp = timestamp
        x = int(x_str)
        y = int(y_str)
        polarity = bool(polarity_str)
        events_list.append((timestamp, x, y, polarity))
        events_store.push_back(timestamp, x, y, polarity)
        if timestamp-first_timestamp>1000000:
            first_timestamp = timestamp
            timestamps.append(timestamp)
            c += 1

writer.writeEvents(events_store)

for t in timestamps:
    # Write 1 dummy image frame (otherwise aedat4to2 doesn't work)
    image = np.full((resolution[0], resolution[1], 3), fill_value=255, dtype=np.uint8)
    frame = dv.Frame(t, image)
    writer.writeFrame(frame)

### then run aedat4to2 events_from_loihi.aedat4 --no_imu --no_frame   

In [11]:
timestamp

1749634048202322

In [14]:
180/c

0.00243509787740635

In [19]:
c

179